# Sentence Selection

Holy Lovenia / 13515113

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data Preparation

In [57]:
from nltk import RegexpTokenizer
from pandas.io.json import json_normalize


import json
import pandas as pd
import re

### Read original data

In [17]:
data = []
with open('dataset/SQuAD/train-v2.0.json') as f:
    json_data = json.load(f)['data']

    for i in range(len(json_data)):
        json_data_i = json_data[i]['paragraphs']
        
        for j in range(1):
            paragraph = json_data_i[j]['context']
                
            data.append(paragraph)

### Preprocess data

In [18]:
preprocessed_data = []
with open('dataset/SQuAD/train-v2.0.json') as f:
    json_data = json.load(f)['data']

    for i in range(len(json_data)):
        json_data_i = json_data[i]['paragraphs']
        
        for j in range(1):
            paragraph = json_data_i[j]['context']
            
            # replace all dictionary phonetic with ''
            paragraph = re.sub('\/.*\ˈ.*\/', '', paragraph)
            
            # replace all japanese characters with ''
            paragraph = re.sub('[\u3000-\u303f\u3040-\u309f\u30a0-\u30ff\uff00-\uff9f\u4e00-\u9faf\u3400-\u4dbf]+', '', paragraph)
            
            # replace dots in the center of words
            words = paragraph.split(' ')
            for i in range(len(words)):
                if(words[i].find('.') != len(words[i]) - 1 and words[i].find('.') != -1):
                    words[i] = words[i].replace('.', '')
                if(words[i].find(',') != len(words[i]) - 1 and words[i].find(',') != -1):
                    words[i] = words[i].replace(',', '')

            paragraph = ' '.join(words)
            
            data_i_j = paragraph.split('.')
            
            paragraph = []
            for k in range(len(data_i_j)):
                tokenizer = RegexpTokenizer('[\w\/\&\-\:]+', flags=re.UNICODE)
    
                token_list = tokenizer.tokenize(data_i_j[k])
                token_list = [token for token in token_list if len(token) > 1 or token.lower() == 'a']
            
                if token_list != []:
                    paragraph.append(token_list)
                
            preprocessed_data.append(paragraph)

## Text Summarization

In [58]:
from sentence_selection.text_rank import TextRank
from sentence_selection.multi_word_phrase_extraction import MultiWordPhraseExtractor

### TextRank

In [51]:
tr = TextRank()

### MultiWordPhraseExtraction

In [52]:
mwpe = MultiWordPhraseExtractor()

### Usage example

In [56]:
count_same_results = 0

for i in range(len(preprocessed_data)):
    multiword_result = mwpe.get_important_sentences(preprocessed_data[i])
    rank_text_result = tr.rank_text(preprocessed_data[i])
    
    if (multiword_result == rank_text_result) or (set(multiword_result).issubset(set(rank_text_result))) or (set(rank_text_result).issubset(set(multiword_result))):
        count_same_results = count_same_results + 1
        same = True
    else:
        same = False
    
    print(i, same, multiword_result, rank_text_result)
    print(i, mwpe.phrases)
    
print('count same results: {} out of {}'.format(count_same_results, len(preprocessed_data)))

0 True [0] [3, 0]
0 ['record American', 'record producer', 'record singer', 'record songwriter']
1 False [0] [2, 1]
1 ['fʁedeʁik Chopin', 'fʁedeʁik fʁɑ', 'fʁedeʁik pronunciation', 'fʁedeʁik French']
2 False [2] [1, 3]
2 ['Tibetan various', 'acceptance Tibetans', 'acceptance full']


/home/holy/Projects/question-answer-generation/sentence_selection/text_rank.py:67: RuntimeWarning: invalid value encountered in true_divide
  similarity_matrix[i] /= similarity_matrix[i].sum()


3 True [0] [0, 2]
3 ['media multi-purpose', 'media players', 'media portable', 'pocket multi-purpose']
4 True [0] [4, 0]
4 ['Densetsu Hepburn', 'Hepburn Princess', 'Densetsu Zeruda']
5 False [2] [1, 3]
5 ['Logan Robert', 'Logan Purvis', 'Logan John']
6 True [0] [0]
6 ['Standard China', 'Standard Time', 'Standard PM', 'Standard epicenter']
7 True [1] [0, 1]
7 ['finance commerce', 'research education', 'finance media']
8 False [1] [2, 0]
8 ['plot literature', 'plot American', 'American classic', 'literature modern']
9 True [0] [0]
9 ['heating such', 'such technologies', 'heating solar', 'photovoltaics heating']
10 True [0] [1, 0]
10 ['designer producer', 'record artist', 'record hop']
11 True [0] [1, 0]
11 ['dharma धर', 'Pali dharma', 'dharma Sanskrit', 'philosophy dharma']
12 True [3] [2, 3]
12 ['Phillips Candice', 'Phillips McCreery', 'Phillips Phillip']
13 True [0] [0]
13 ['behaviors millennia', 'attributes physical']
14 False [0, 2] [0, 1]
14 ['torch run', 'torch Olympics', 'torch re

93 True [0, 1] [1, 0]
93 ['element least', 'element processing', 'CPU unit']
94 False [2] [0, 1]
94 ['many communities', 'many individuals', 'many context', 'communities black']
95 True [1] [2, 1]
95 ['Register Daily', 'Register Universal']
96 True [0, 1] [1, 0]
96 ['seat government', 'government India', 'addition India', 'seat capital']
97 False [3] [2, 1]
97 ['barriers natural', 'barriers routes', 'barriers such', 'natural specific']
98 False [1, 2] [1, 0]
98 ['Township Harbor', 'Brigantine city', 'Township Galloway']
99 True [2] [2, 0]
99 ['oncology virology', 'oncology transplantation', 'oncology organ', 'virology psychiatry']
100 True [0] [1, 0]
100 ['MPEG-1 MPEG-2', 'MPEG-1 Audio', 'Layer III']
101 False [2] [0, 3]
101 ['Detroit such', 'Detroit cities', 'Detroit New', 'other such']
102 False [1] [2, 0]
102 ['shallow drawers', 'shallow type', 'cases hold']
103 False [0, 2] [3, 1]
103 ['state nomad', 'state tribes', 'name tribes', 'state part']
104 True [0] [0, 1]
104 ['al-Din Usul

181 False [6] [5, 1]
181 ['league average', 'league highest', 'league position', 'century highest']
182 False [1] [0, 3]
182 ['rash-causing surfaces', 'rash-causing insect', 'rash-causing plants', 'insect splinters']
183 False [0, 1, 2] [2, 3]
183 ['American professional', 'American team', 'professional located', 'American Central']
184 False [0] [3, 2]
184 ['Chosungul North', 'Chosungul Haebang', 'Chosungul 조국해방전쟁', 'Haebang Fatherland']
185 True [0] [0, 2]
185 ['certain permission', 'exclusive permission', 'permission copyright', 'certain rights']
186 True [1] [1, 0]
186 ['shares land', 'shares peninsula', 'shares borders', 'Balkan shares']
187 True [0] [0, 2]
187 ['petrochemicals power', 'refining exploration', 'petrochemicals generation']
188 False [0] [2, 1]
188 ['primates elephants', 'elephants such', 'elephants humans', 'elephants intelligent']
189 False [0] [1, 2]
189 ['Elizabeth petition', 'Elizabeth Queen', 'Elizabeth permission', 'merchants petition']
190 True [0] [0, 1]
190

268 True [1] [1, 2]
268 ['Winchester Radley', 'Radley others', 'Winchester Harrow', 'United others']
269 True [2] [3, 2]
269 ['global network', 'global trade', 'global Scandinavian']
270 False [3] [1, 2]
270 ['Cortegada Islands', 'Cortegada Ons', 'Cortegada Sálvora', 'Islands Cíes']
271 True [0, 2] [2, 0]
271 ['digital devices', 'digital printers', 'digital portable']
272 True [0] [0, 1]
272 ['North China', 'North dynasties', 'Sichuan North', 'North Zhou']
273 True [2] [2, 3]
273 ['decomposition collation', 'decomposition rules', 'rules properties', 'properties such']
274 False [0, 2, 4] [2, 1]
274 ['United States', 'United border', 'United Canada', 'United area']
275 False [5] [1, 0]
275 ['conurbation Greater', 'conurbation note', 'conurbation forms', 'note region']
276 True [0] [0]
276 ['time Theory', 'time Management', 'Theory posits', 'time Terror']
277 True [0] [3, 0]
277 ['الكبرى الصحراء', 'الكبرى aṣ-ṣaḥrāʾ', 'الكبرى al-kubrā', 'الكبرى Arabic']
278 False [0, 1] [1, 2]
278 ['offic

355 False [2, 3] [3, 4]
355 ['Hispanophone largest', 'Hispanophone country', 'speakers world']
356 True [0] [1, 0]
356 ['Metropolitan Area', 'Metropolitan Summerville', 'Metropolitan Statistical', 'North Metropolitan']
357 True [2] [2, 0]
357 ['bombing Churchill', 'bombing retaliatory', 'Churchill Prime']
358 True [1] [0, 1]
358 ['untempered development', 'untempered adequate', 'untempered growth', 'concern untempered']
359 False [1, 5] [4, 5]
359 ['adjective Latin', 'Latin term', 'adjective vacuus', 'adjective vacant']
360 True [0, 1, 2, 3, 4] [3, 4]
360 ['Chinese cháo', 'Chinese dynasty', 'Chinese Han', 'Chinese Hàn']
361 True [1] [1, 2]
361 ['divine series', 'messages series', 'culmination series']
362 True [2] [2, 0]
362 ['United Nations', 'United Division', 'United Statistics']
363 False [0, 1, 2] [0, 3]
363 ['digital disc', 'storage format', 'digital data']
364 True [0] [2, 0]
364 ['power electrical', 'power signals', 'electronic switch']
365 False [0] [3, 1]
365 ['people era', '

441 True [0] [0, 2]
441 ['other energy', 'other such', 'such light', 'other phenomena']
count same results: 207 out of 442


In [20]:
idx = 121

for i in range(len(preprocessed_data[idx])):
    print(i, ' '.join(preprocessed_data[idx][i]))

0 Emotions are complex
1 According to some theories they are a state of feeling that results in physical and psychological changes that influence our behavior
2 The physiology of emotion is closely linked to arousal of the nervous system with various states and strengths of arousal relating apparently to particular emotions
3 Emotion is also linked to behavioral tendency
4 Extroverted people are more likely to be social and express their emotions while introverted people are more likely to be more socially withdrawn and conceal their emotions
5 Emotion is often the driving force behind motivation positive or negative
6 Definition has been described as is a positive or negative experience that is associated with a particular pattern of physiological activity According to other theories emotions are not causal forces but simply syndromes of components which might include motivation feeling behavior and physiological changes but no one of these components is the emotion
7 Nor is the emoti